In [96]:
from sklearn.model_selection import train_test_split
import pandas as pd
import keras
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [97]:
#In A RNN you want to predict the next values 
df = pd.read_csv("Month_Value_1.csv")
print(df.head())

       Period       Revenue  Sales_quantity  Average_cost  \
0  01.01.2015  1.601007e+07         12729.0   1257.763541   
1  01.02.2015  1.580759e+07         11636.0   1358.507000   
2  01.03.2015  2.204715e+07         15922.0   1384.697024   
3  01.04.2015  1.881458e+07         15227.0   1235.606705   
4  01.05.2015  1.402148e+07          8620.0   1626.621765   

   The_average_annual_payroll_of_the_region  
0                                30024676.0  
1                                30024676.0  
2                                30024676.0  
3                                30024676.0  
4                                30024676.0  


In [98]:
#Eliminate Nans in the data Set
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)


/var/folders/64/_d7wbs111535lmdv26qfwp000000gn/T/ipykernel_37631/3841858164.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/var/folders/64/_d7wbs111535lmdv26qfwp000000gn/T/ipykernel_37631/3841858164.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [99]:
values = df.iloc[:, 1:5].values
features = values.shape[1]
print(f"the features that the model wants to learn: {features}")
values_of_nans = df.isna().sum().sum()
values_of_total = df.size
print(f"Total of Nans: {values_of_nans / values_of_total}")


the features that the model wants to learn: 4
Total of Nans: 0.0


In [104]:
scaler = MinMaxScaler(feature_range=(0,1))
values_normalize = scaler.fit_transform(values)

X, y = [], []
window_size = 10
for i in range(len(values_normalize) - window_size):
    X.append(values_normalize[i: i+window_size])
    y.append(values_normalize[i + window_size])

X = np.array(X)           
y = np.array(y)          
X = X.astype(np.float32)
y = y.astype(np.float32)


In [108]:
optimizer = keras.optimizers.SGD(learning_rate=5e-5, clipnorm=1.0)

model = keras.Sequential([
  keras.layers.LSTM(100, activation="tanh", return_sequences=True, input_shape=(X.shape[1], X.shape[2])), #Return all the sequences
  keras.layers.LSTM(50 , activation="tanh"),
  keras.layers.Dense(1)  
])

model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.summary()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_20 (LSTM)                  │ (None, 10, 100)        │        42,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_21 (LSTM)                  │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,251 (282.23 KB)

 Trainable params: 72,251 (282.23 KB)

 Non-trainable params: 0 (0.00 B)

In [109]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.30, shuffle=False
)
print(f"X train shape: {X_train.shape}")
print(f"Y train shape: {Y_train.shape}")
print(f"X test shape: {X_test.shape}")
print(f"Y test shape: {Y_test.shape}")


X train shape: (60, 10, 4)
Y train shape: (60, 4)
X test shape: (26, 10, 4)
Y test shape: (26, 4)


In [111]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=100)
model.compile

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - loss: 0.3140 - mae: 0.4914 - val_loss: 0.5403 - val_mae: 0.7272
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.3135 - mae: 0.4909 - val_loss: 0.5395 - val_mae: 0.7266
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.3130 - mae: 0.4905 - val_loss: 0.5386 - val_mae: 0.7261
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.3125 - mae: 0.4900 - val_loss: 0.5378 - val_mae: 0.7255
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 0.3121 - mae: 0.4895 - val_loss: 0.5370 - val_mae: 0.7249
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.3116 - mae: 0.4890 - val_loss: 0.5362 - val_mae: 0.7244
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.3111 - mae: 0.4886 - val_loss: 0.5354 - val_mae: 0.7238
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.3106 - mae: 0.4881 - val_loss: 0.5346 - val_mae: 0.7233
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 0.3102 - ma

<bound method Trainer.compile of <Sequential name=sequential_10, built=True>>

In [ ]:
loss, mae = model.evaluate(X_test, Y_test)
pri